In [53]:


import numpy as np
import tensorflow as tf

from drowsiness_detection import config
from drowsiness_detection.models import build_dummy_tf_classifier, ThreeDStandardScaler,build_dense_model, build_lstm_model, build_cnn_model
import tensorflow.keras as keras
import math

In [2]:
window_size = 10
recording_frequency = 30
exclude_by = "a"

config.set_paths(recording_frequency, window_size)
config.PATHS.WINDOW_DATA

PosixPath('/home/tim/Windows/Windows_30_Hz/WindowData/10_sec')

In [16]:
num_samples = 20
X = np.random.random(num_samples * 300 * 7).reshape((num_samples, 300, 7))
y = np.concatenate((np.zeros((num_samples // 2)), np.ones((num_samples // 2))))
print(X.shape, y.shape)

(20, 300, 7) (20,)


In [17]:
scaler = ThreeDStandardScaler()
X_scaled = scaler.fit_transform(X, y)
print(X_scaled.shape)

(20, 300, 7)


In [18]:
from collections import namedtuple

Input = namedtuple("Input", "X y")
input = Input(X_scaled, y)

In [59]:
# learning rate schedule
def step_decay(epoch):
	initial_lrate = 0.01
	drop = 0.5
	epochs_drop = 10.0
	lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
	return lrate


def test_model(model, input: Input = input):
    batch_size = 2
    epochs = 2
    X = tf.convert_to_tensor(input.X, dtype=tf.float64)
    y = tf.convert_to_tensor(input.y, dtype=tf.int8)

    lrate = keras.callbacks.LearningRateScheduler(step_decay)
    callbacks_list = [lrate]

    model.fit(X, y, batch_size=batch_size, epochs=epochs, validation_split=0.1, callbacks=callbacks_list)
    score = model.evaluate(X_scaled, y)
    print("Test loss:", score[0])
    print("Test accuracy:", score[1])

In [20]:
test_model(model=build_dummy_tf_classifier(input_shape=X.shape))

Activation was set to None which is not supported.
Epoch 1/2
9/9 [==============================] - 0s 17ms/step - loss: 6.4064 - accuracy: 0.3333 - val_loss: 0.3278 - val_accuracy: 1.0000
Epoch 2/2
1/1 [==============================] - 0s 91ms/step - loss: 5.7986 - accuracy: 0.4000
Test loss: 5.798554420471191
Test accuracy: 0.4000000059604645


In [22]:
test_model(model=build_dense_model(input_shape=X.shape))

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_3 (Flatten)         (None, 2100)              0         
                                                                 
 dense_5 (Dense)             (None, 64)                134464    
                                                                 
 dense_6 (Dense)             (None, 1)                 65        
                                                                 
Total params: 134,529
Trainable params: 134,529
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/2
9/9 [==============================] - 0s 14ms/step - loss: 0.9065 - accuracy: 0.4444 - val_loss: 0.4561 - val_accuracy: 1.0000
Epoch 2/2
1/1 [==============================] - 0s 88ms/step - loss: 0.0449 - accuracy: 0.5000
Test loss: 0.04489997401833534
Test accuracy: 0.5


In [23]:
def build_cnn_model(input_shape,
                    kernel_size=(5, 1),
                    stride=(3, 1),
                    num_filters=32,
                    padding="valid",
                    use_batch_norm=True,
                    lr=.001):
    input_layer = keras.layers.Input(input_shape[1:])
    try:
        reshape_layer = keras.layers.Reshape(input_shape[1:] + [1,])(input_layer)
    except TypeError:
        reshape_layer = keras.layers.Reshape(input_shape[1:] + (1,))(input_layer)

    conv1 = keras.layers.Conv2D(filters=num_filters, kernel_size=kernel_size, strides=stride,
                                padding=padding)(reshape_layer)
    if use_batch_norm:
        conv1 = keras.layers.BatchNormalization()(conv1)
    conv1 = keras.layers.ReLU()(conv1)

    conv2 = keras.layers.Conv2D(filters=num_filters, kernel_size=kernel_size, padding=padding)(
        conv1)
    if use_batch_norm:
        conv2 = keras.layers.BatchNormalization()(conv2)
    conv2 = keras.layers.ReLU()(conv2)

    conv3 = keras.layers.Conv2D(filters=num_filters, kernel_size=kernel_size, padding=padding)(
        conv2)
    if use_batch_norm:
        conv3 = keras.layers.BatchNormalization()(conv3)
    conv3 = keras.layers.ReLU()(conv3)

    gap = keras.layers.GlobalAveragePooling2D()(conv3)

    output_layer = keras.layers.Dense(1, activation="sigmoid")(gap)
    model = keras.models.Model(inputs=input_layer, outputs=output_layer)
    # Compile model
    # callbacks = [
    # keras.callbacks.ReduceLROnPlateau(
    #     monitor="val_loss", factor=0.5, patience=20, min_lr=0.0001
    # ),
    # keras.callbacks.EarlyStopping(monitor="val_loss", patience=50, verbose=1),
    # ]
    optimizer = keras.optimizers.Adam(learning_rate=lr)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    print(model.summary())
    return model

In [57]:
def build_cnn_model_1d(input_shape,
                    kernel_size=5,
                    stride=1,
                    num_filters=32,
                    num_conv_layers=2,
                    padding="same",
                    use_batch_norm=True,
                    lr=0.1,
                    pooling="average"):
    input_layer = keras.layers.Input(input_shape[1:])
    prev_layer = input_layer
    for _ in range(num_conv_layers):
        conv_layer = keras.layers.Conv1D(filters=num_filters, kernel_size=kernel_size, strides=stride,
                                    padding=padding)(prev_layer)
        if use_batch_norm:
            conv_layer = keras.layers.BatchNormalization()(conv_layer)
        conv_layer = keras.layers.ReLU()(conv_layer)
        prev_layer = conv_layer

    if pooling == "average":
        pool_layer = keras.layers.GlobalAveragePooling1D()(prev_layer)
    elif pooling == "max":
        pool_layer = keras.layers.GlobalMaxPool1D()(prev_layer)
    else:
        pool_layer = prev_layer

    output_layer = keras.layers.Dense(1, activation="sigmoid")(pool_layer)

    model = keras.models.Model(inputs=input_layer, outputs=output_layer)
    optimizer = keras.optimizers.Adam(learning_rate=lr)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    print(model.summary())
    return model

In [58]:
test_model(model=build_cnn_model_1d(input_shape=X.shape, pooling="max"))

Model: "model_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_22 (InputLayer)       [(None, 300, 7)]          0         
                                                                 
 conv1d_22 (Conv1D)          (None, 300, 32)           1152      
                                                                 
 batch_normalization_28 (Bat  (None, 300, 32)          128       
 chNormalization)                                                
                                                                 
 re_lu_28 (ReLU)             (None, 300, 32)           0         
                                                                 
 conv1d_23 (Conv1D)          (None, 300, 32)           5152      
                                                                 
 batch_normalization_29 (Bat  (None, 300, 32)          128       
 chNormalization)                                         

In [14]:
test_model(model=build_lstm_model(input_shape=X.shape))

Epoch 1/2
9/9 [==============================] - 4s 251ms/step - loss: 0.7312 - accuracy: 0.3889 - val_loss: 0.8542 - val_accuracy: 0.0000e+00
Epoch 2/2
1/1 [==============================] - 1s 842ms/step - loss: 0.2074 - accuracy: 0.9000
Test loss: 0.20741109549999237
Test accuracy: 0.8999999761581421


use real data